# Import Libraries

In [1]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

/home/codespace/.phi3_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Read Document

In [2]:
# Load pdf documents
documents_1 = ''

reader = PdfReader('/workspaces/digital_research_guide/1706.03762v7.pdf')
for page in reader.pages:
    documents_1 += page.extract_text()

In [3]:
print(documents_1)

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experime

In [5]:
type(documents_1)

str

In [6]:
cleaned_string = documents_1.replace('\n', ' ')
cleaned_string

'Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need Ashish Vaswani∗ Google Brain avaswani@google.comNoam Shazeer∗ Google Brain noam@google.comNiki Parmar∗ Google Research nikip@google.comJakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Research llion@google.comAidan N. Gomez∗ † University of Toronto aidan@cs.toronto.eduŁukasz Kaiser∗ Google Brain lukaszkaiser@google.com Illia Polosukhin∗ ‡ illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experim

In [7]:
print(cleaned_string)

Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works. Attention Is All You Need Ashish Vaswani∗ Google Brain avaswani@google.comNoam Shazeer∗ Google Brain noam@google.comNiki Parmar∗ Google Research nikip@google.comJakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Research llion@google.comAidan N. Gomez∗ † University of Toronto aidan@cs.toronto.eduŁukasz Kaiser∗ Google Brain lukaszkaiser@google.com Illia Polosukhin∗ ‡ illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experime

In [12]:
sentence_list = cleaned_string.split('. ')

In [14]:
len(sentence_list)

371

# Splitting Document

In [ ]:
# Document Splitting
chunk_size = 20000
chunk_overlap = 5

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
split = splitter.split_text(documents_1)

In [ ]:
len(split)

# Create Embedding

In [ ]:
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

In [ ]:
def get_embeddings(doc):
    model_input = doc
    out =  model.encode(model_input)
    return out

In [ ]:
x = list(map(get_embeddings,split))

In [ ]:
import numpy as np
x_array = np.array(x)

[Indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes)

In [ ]:
import faiss
index = faiss.IndexFlatL2(1024)
index.add(x_array)

In [ ]:
faiss.write_index(index, 'test.index')